# 환경 설정

In [1]:
# 한글 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 메뉴 - 런타임 - 세션 다시 시작

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 1 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (4,385 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 117528 files and direc

In [2]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.4 MB/s eta 0:00:00


# 모듈 설치

In [1]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [157]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, t, zscore
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

In [3]:
# 한글 폰트 지정
plt.rc("font", family = "NanumGothic")

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
file_path = '/content/drive/MyDrive/MedicalDA04/Semi Project#1/Analysis Dataset/1. 기초 분석/'
file_name = 'COVID19_Medical_Infra_Inc_2021_2022_v1_w_DV_IDV.csv'

# 기본 선형 회귀 모델 (감염률)

In [46]:
df = pd.read_csv(file_path+file_name, header = 0, encoding = ' euc-kr')
df.head(1)

,시도명,시도명 코드,연도,월별,COVID19 확진자 수,COVID19 사망자 수,COVID19 1차 예방 접종자 수,COVID19 2차 예방 접종자 수,COVID19 3차 예방 접종자 수,COVID19 총 예방 접종자 수,총인구(명),총 고령인구(명),상급종합병원,종합병원,병원,의원,보건의료기관,의료인프라 총합,총 의사수(명),총 간호사수(명),총 의료인력 수(명),총 일반입원실 병실수,총 일반입원실 병상수,총 중환자실 병실수,총 중환자실 병상수,총 격리병실 병실수,총 격리병실 병상수,총 무균치료실 병실수,총 무균치료실 병상수,총 의료기관 병실수,총 의료기관 병상수,1인당 개인소득,1인당 지역총소득,감염률_per100k,사망률_per100k,치명률_percent,3차 접종률_percent,총 의사수_per100k,총 간호사수_per100k,총 의료인력 수_per100k,총 상급종합병원_percent,총 종합병원_percent,총 병원_percent,총 의원_percent,총 보건의료기관_percent,총 일반입원실 병실수_percent,총 중환자실 병실수_percent,총 격리병실 병실수_percent,총 무균치료실 병실수_percent,총 일반입원실 병상수_percent,총 중환자실 병상수_percent,총 격리병실 병상수_percent,총 무균치료실 병상수_percent,총 고령인구수_per100k,1인당 개인소득_perIndiv,1인당 지역총소득_perIndiv
0,서울,11,2021,1,5159,147,0,0,0,0,9472127,1564390,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,54.465064,1.551922,2.849389,0.0,338.308386,620.483657,958.792043,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,16515.720281,27119,59498


In [47]:
df1 = df.copy()
columns_eng = [
    "region", "region_code", "year", "month",
    "confirmed", "deaths",
    "vax1", "vax2", "vax3", "vax_total",
    "pop_total", "pop_elderly",
    "tertiary_hosp", "general_hosp", "hospital", "clinic", "public_health",
    "infra_total",
    "docs_total", "nurses_total", "staff_total",
    "ward_rooms", "ward_beds",
    "icu_rooms", "icu_beds",
    "iso_rooms", "iso_beds",
    "sterile_rooms", "sterile_beds",
    "rooms_total", "beds_total",
    "income_indiv", "region_income_indiv",
    "confirmed_per100k", "death_per100k", "crit_pct", "vax3_pct",
    "docs_total_per100k", "nurses_total_per100k", "staff_total_per100k",
    "tertiary_hosp_pct", "general_hosp_pct", "hospital_pct", "clinic_pct", "public_health_pct",
    "ward_rooms_pct", "icu_rooms_pct", "iso_rooms_pct", "sterile_rooms_pct",
    "ward_beds_pct", "icu_beds_pct", "iso_beds_pct", "sterile_beds_pct",
    "pop_elderly_per100k", "income_perindiv", "region_income_perindiv"
]

df1.columns = columns_eng

In [48]:
df1.head(1)

,region,region_code,year,month,confirmed,deaths,vax1,vax2,vax3,vax_total,pop_total,pop_elderly,tertiary_hosp,general_hosp,hospital,clinic,public_health,infra_total,docs_total,nurses_total,staff_total,ward_rooms,ward_beds,icu_rooms,icu_beds,iso_rooms,iso_beds,sterile_rooms,sterile_beds,rooms_total,beds_total,income_indiv,region_income_indiv,confirmed_per100k,death_per100k,crit_pct,vax3_pct,docs_total_per100k,nurses_total_per100k,staff_total_per100k,tertiary_hosp_pct,general_hosp_pct,hospital_pct,clinic_pct,public_health_pct,ward_rooms_pct,icu_rooms_pct,iso_rooms_pct,sterile_rooms_pct,ward_beds_pct,icu_beds_pct,iso_beds_pct,sterile_beds_pct,pop_elderly_per100k,income_perindiv,region_income_perindiv
0,서울,11,2021,1,5159,147,0,0,0,0,9472127,1564390,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,54.465064,1.551922,2.849389,0.0,338.308386,620.483657,958.792043,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,16515.720281,27119,59498


In [49]:
df1.describe()

,region_code,year,month,confirmed,deaths,vax1,vax2,vax3,vax_total,pop_total,pop_elderly,tertiary_hosp,general_hosp,hospital,clinic,public_health,infra_total,docs_total,nurses_total,staff_total,ward_rooms,ward_beds,icu_rooms,icu_beds,iso_rooms,iso_beds,sterile_rooms,sterile_beds,rooms_total,beds_total,income_indiv,region_income_indiv,confirmed_per100k,death_per100k,crit_pct,vax3_pct,docs_total_per100k,nurses_total_per100k,staff_total_per100k,tertiary_hosp_pct,general_hosp_pct,hospital_pct,clinic_pct,public_health_pct,ward_rooms_pct,icu_rooms_pct,iso_rooms_pct,sterile_rooms_pct,ward_beds_pct,icu_beds_pct,iso_beds_pct,sterile_beds_pct,pop_elderly_per100k,income_perindiv,region_income_perindiv
count,408.000000,408.000000,408.000000,4.080000e+02,408.000000,4.080000e+02,4.080000e+02,4.080000e+02,4.080000e+02,4.080000e+02,4.080000e+02,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000,408.000000
mean,29.176471,2021.500000,6.500000,7.102954e+04,76.573529,1.050183e+05,1.040353e+05,7.837673e+04,2.874303e+05,3.042069e+06,5.250897e+05,2.647059,19.029412,82.205882,2025.588235,203.588235,2333.058824,6537.000000,14545.117647,21082.117647,10347.882353,37516.941176,62.058824,661.941176,162.000000,348.382353,15.764706,24.764706,10587.705882,38552.029412,23756.882353,41010.852941,2306.588545,2.376829,0.504007,2.609220,197.445224,471.418554,668.863778,0.108463,0.981189,3.665570,82.783698,12.461079,97.719492,0.582457,1.568071,0.129980,97.280531,1.763114,0.901114,0.055241,17797.565931,23756.882353,41010.852941
std,7.311143,0.500614,3.456291,2.068720e+05,178.979593,2.885880e+05,3.087950e+05,2.948347e+05,6.309750e+05,3.299463e+06,4.887924e+05,3.202109,14.607363,71.888439,2442.355299,197.037287,2518.769782,8157.579186,15486.404651,23612.090345,8409.972942,29922.829205,72.860922,695.081192,153.706252,339.959993,31.223747,49.787988,8634.942860,30851.702831,1760.028787,8060.557802,4146.676364,4.150150,1.167236,6.607729,54.528662,114.739035,164.255103,0.067795,0.338802,1.612459,11.320118,11.337643,1.027037,0.315656,0.716006,0.126802,1.153258,0.745478,0.491761,0.057546,3680.260337,1760.028787,8060.557802
min,11.000000,2021.000000,1.000000,3.000000e+01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.662270e+05,3.572700e+04,0.000000,2.000000,1.000000,198.000000,16.000000,219.000000,458.000000,882.000000,1340.000000,659.000000,1930.000000,2.000000,26.000000,6.000000,7.000000,0.000000,0.000000,669.000000,1976.000000,21251.000000,31558.000000,3.036909,0.000000,0.000000,0.000000,125.059048,240.834237,365.893285,0.000000,0.429228,0.421941,59.272951,0.347368,94.544096,0.243814,0.640697,0.000000,94.284514,1.001010,0.262467,0.000000,9755.424914,21251.000000,31558.000000
25%,24.000000,2021.000000,3.750000,6.472500e+02,3.000000,3.642500e+02,3.792500e+02,0.000000e+00,3.006000e+03,1.475262e+06,2.262240e+05,1.000000,11.000000,37.000000,912.000000,24.000000,1093.000000,2778.000000,6647.000000,9431.000000,4798.000000,17160.000000,22.000000,349.000000,81.000000,154.000000,3.000000,4.000000,4919.000000,17732.000000,22338.000000,35422.000000,36.742277,0.168693,0.068235,0.000000,159.624824,386.253782,538.434795,0.061387,0.751252,2.882600,72.676580,1.585977,97.239228,0.349752,1.002770,0.061291,96.774194,1.269774,0.427718,0.023447,14926.203252,22338.000000,35422.000000
50%,31.000000,2021.500000,6.500000,6.741000e+03,17.500000,4.670500e+03,3.190000e+03,1.264500e+03,4.133950e+04,1.782588e+06,4.125980e+05,2.000000,13.000000,57.500000,1117.500000,65.000000,1626.000000,3669.500000,9094.000000,12463.000000,8971.000000,34965.000000,43.000000,435.500000,106.

In [75]:
# ============================================================================
# 0 값 확인
# ============================================================================
dependent_vars = df1.columns[4:]

print("\n" + "=" * 100)
print("0 값 확인")
print("=" * 100)

zero_analysis = []

for var in dependent_vars:
    # missing_count = df1[var].isnull().sum()
    zero_count = (df1[var]==0).sum()
    zero_percent = (zero_count / len(df1)) * 100

    zero_analysis.append({
        '변수': var,
        '0값 개수': zero_count,
        '0값 비율(%)': zero_percent,
    })

zero_df = pd.DataFrame(zero_analysis)

print("\n0 값 분석\n")
print(zero_df.to_string(index=False))
# zero_df.to_excel(file_path + 'zero_analysis.xlsx', sheet_name='zero_analysis', index=True)


0 값 확인

0 값 분석

                    변수  0값 개수  0값 비율(%)
             confirmed      0  0.000000
                deaths     37  9.068627
                  vax1     17  4.166667
                  vax2     34  8.333333
                  vax3    153 37.500000
             vax_total     17  4.166667
             pop_total      0  0.000000
           pop_elderly      0  0.000000
         tertiary_hosp     72 17.647059
          general_hosp      0  0.000000
              hospital      0  0.000000
                clinic      0  0.000000
         public_health      0  0.000000
           infra_total      0  0.000000
            docs_total      0  0.000000
          nurses_total      0  0.000000
           staff_total      0  0.000000
            ward_rooms      0  0.000000
             ward_beds      0  0.000000
             icu_rooms      0  0.000000
              icu_beds      0  0.000000
             iso_rooms      0  0.000000
              iso_beds      0  0.000000
         sterile_rooms 

In [51]:
X_vars = df1.loc[:,'tertiary_hosp':'region_income_indiv'].columns.tolist() + ['pop_elderly']
Y_var_name = 'confirmed_per100k'
Y_var = [Y_var_name]

Y_log_var_name = 'log_confirmed_per100k'

In [52]:
df2 = df1.loc[:, Y_var + X_vars]
df2.head(1)

,confirmed_per100k,tertiary_hosp,general_hosp,hospital,clinic,public_health,infra_total,docs_total,nurses_total,staff_total,ward_rooms,ward_beds,icu_rooms,icu_beds,iso_rooms,iso_beds,sterile_rooms,sterile_beds,rooms_total,beds_total,income_indiv,region_income_indiv,pop_elderly
0,54.465064,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,1564390


In [53]:
# Y1: 감염률 per100k ~ 원시 독립변수
formula_X = ' + '.join(X_vars)
formula_Y1 = f"{Y_var_name} ~ {formula_X}"
results_Y1_ori = smf.ols(formula=formula_Y1, data=df2).fit()

print("### OLS 모델 결과 (Y1: 변화율 그대로) ###")
print(results_Y1_ori.summary())
# R-squared 결정계수 = 설명력
# Adj. R-squared 조정결정계수 = 독립변수의 개수가 다른 모델끼리 비교할 때 사용
# F 검정 결과 = 회귀모델 유의성 검정
# AIC 모델 적합성 비교할 때 사용
# 절편 - 유의함
# 기울기 - 유의함
# Durbin-Watson 2에 가까울수록 오차항이 독립

### OLS 모델 결과 (Y1: 변화율 그대로) ###
                            OLS Regression Results                            
Dep. Variable:      confirmed_per100k   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     6.714
Date:                Thu, 18 Dec 2025   Prob (F-statistic):           9.81e-15
Time:                        00:35:56   Log-Likelihood:                -3921.9
No. Observations:                 408   AIC:                             7882.
Df Residuals:                     389   BIC:                             7958.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
In

In [54]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
df2_log = df2.copy()

# 1. 컬럼별로 -inf(로그 0) 대신 넣고 싶은 '로그 값' 지정
# 주의: 이 값은 로그가 취해진 상태의 값입니다. (예: 0, -1, -5 등)
custom_log_replace_values = {
    'tertiary_hosp': 0,   # 로그 변환 후 -inf라면 0.1으로 대체
    'sterile_rooms': 0,
    'sterile_beds': 0
}

# 기본 대체 로그값 (딕셔너리에 없는 컬럼들이 사용할 값, 로그 1 = 0 이므로 0.0 권장)
default_log_value = 0.0

# 원본 컬럼 리스트 고정
original_columns = df2_log.columns.tolist()

# 2. 순회 및 변환
for col in original_columns:
    # (A) 먼저 로그 변환 수행 (0인 지점은 -inf가 됨)
    # RuntimeWarning을 무시하기 위해 errstate 사용
    with np.errstate(divide='ignore'):
        log_series = np.log(df2_log[col])

    # (B) 해당 컬럼의 대체 로그값 결정
    replacement = custom_log_replace_values.get(col, default_log_value)

    # (C) 결과가 -inf인 것만 찾아서 replacement로 치환
    # np.isinf()를 사용하여 음의 무한대(-inf)를 체크합니다.
    log_series = log_series.replace(-np.inf, replacement)

    # (D) 새로운 컬럼으로 저장
    df2_log[f'log_{col}'] = log_series

# 3. 'log_'가 포함된 컬럼 이름들만 필터링하여 최종 추출
columns_log = [col for col in df2_log.columns if 'log_' in col.lower()]
df2_log = df2_log[columns_log]

print("✅ 로그 변환 후 -inf 값 사후 처리 완료")

✅ 로그 변환 후 -inf 값 사후 처리 완료


In [55]:
df2_log.head(1)

,log_confirmed_per100k,log_tertiary_hosp,log_general_hosp,log_hospital,log_clinic,log_public_health,log_infra_total,log_docs_total,log_nurses_total,log_staff_total,log_ward_rooms,log_ward_beds,log_icu_rooms,log_icu_beds,log_iso_rooms,log_iso_beds,log_sterile_rooms,log_sterile_beds,log_rooms_total,log_beds_total,log_income_indiv,log_region_income_indiv,log_pop_elderly
0,3.997559,2.639057,3.73767,5.402677,9.125762,3.496508,9.159047,10.374896,10.981438,11.416613,10.140258,11.336499,5.521461,7.886081,6.45677,7.186901,4.905275,5.370638,10.179793,11.385353,10.20799,10.993698,14.263007


In [56]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
X_log_vars = columns_log[1:]
formula_X_log = ' + '.join(X_log_vars)
formula_Y1_log = f"{Y_log_var_name} ~ {formula_X_log}"
results_Y1_ori_log = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log.summary())
# R-squared 결정계수 = 설명력
# Adj. R-squared 조정결정계수 = 독립변수의 개수가 다른 모델끼리 비교할 때 사용
# F 검정 결과 = 회귀모델 유의성 검정
# AIC 모델 적합성 비교할 때 사용
# 절편 - 유의함
# 기울기 - 유의함
# Durbin-Watson 2에 가까울수록 오차항이 독립

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.801
Model:                               OLS   Adj. R-squared:                  0.790
Method:                    Least Squares   F-statistic:                     70.39
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):          3.36e-120
Time:                           00:35:57   Log-Likelihood:                -600.03
No. Observations:                    408   AIC:                             1246.
Df Residuals:                        385   BIC:                             1338.
Df Model:                             22                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [57]:
df1.head(1)

,region,region_code,year,month,confirmed,deaths,vax1,vax2,vax3,vax_total,pop_total,pop_elderly,tertiary_hosp,general_hosp,hospital,clinic,public_health,infra_total,docs_total,nurses_total,staff_total,ward_rooms,ward_beds,icu_rooms,icu_beds,iso_rooms,iso_beds,sterile_rooms,sterile_beds,rooms_total,beds_total,income_indiv,region_income_indiv,confirmed_per100k,death_per100k,crit_pct,vax3_pct,docs_total_per100k,nurses_total_per100k,staff_total_per100k,tertiary_hosp_pct,general_hosp_pct,hospital_pct,clinic_pct,public_health_pct,ward_rooms_pct,icu_rooms_pct,iso_rooms_pct,sterile_rooms_pct,ward_beds_pct,icu_beds_pct,iso_beds_pct,sterile_beds_pct,pop_elderly_per100k,income_perindiv,region_income_perindiv
0,서울,11,2021,1,5159,147,0,0,0,0,9472127,1564390,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,54.465064,1.551922,2.849389,0.0,338.308386,620.483657,958.792043,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,16515.720281,27119,59498


In [58]:
df3 = df1.copy()
X_vars = df1.loc[:,'docs_total_per100k':'region_income_perindiv'].columns.tolist()
Y_var = [Y_var_name]

In [59]:
df3 = df3.loc[:, Y_var + X_vars]
df3.head(1)

,confirmed_per100k,docs_total_per100k,nurses_total_per100k,staff_total_per100k,tertiary_hosp_pct,general_hosp_pct,hospital_pct,clinic_pct,public_health_pct,ward_rooms_pct,icu_rooms_pct,iso_rooms_pct,sterile_rooms_pct,ward_beds_pct,icu_beds_pct,iso_beds_pct,sterile_beds_pct,pop_elderly_per100k,income_perindiv,region_income_perindiv
0,54.465064,338.308386,620.483657,958.792043,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,16515.720281,27119,59498


In [60]:
# Y1: 감염률 per100k ~ 파생 독립변수
formula_X = ' + '.join(X_vars)
formula_Y1 = f"{Y_var_name} ~ {formula_X}"
results_Y1_derive = smf.ols(formula=formula_Y1, data=df3).fit()

print("### OLS 모델 결과 (Y1: 변화율 그대로) ###")
print(results_Y1_derive.summary())
# R-squared 결정계수 = 설명력
# Adj. R-squared 조정결정계수 = 독립변수의 개수가 다른 모델끼리 비교할 때 사용
# F 검정 결과 = 회귀모델 유의성 검정
# AIC 모델 적합성 비교할 때 사용
# 절편 - 유의함
# 기울기 - 유의함
# Durbin-Watson 2에 가까울수록 오차항이 독립

### OLS 모델 결과 (Y1: 변화율 그대로) ###
                            OLS Regression Results                            
Dep. Variable:      confirmed_per100k   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     7.809
Date:                Thu, 18 Dec 2025   Prob (F-statistic):           2.12e-15
Time:                        00:35:57   Log-Likelihood:                -3923.8
No. Observations:                 408   AIC:                             7880.
Df Residuals:                     392   BIC:                             7944.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

In [61]:
# Log Y1: 감염률 per100k (로그화) ~ 파생 독립변수 (일부 로그화)
df3_log = df1.copy()

# 1. 제외하고 싶은 컬럼 리스트 정의
# Correct approach to select a range of columns
start_col = 'tertiary_hosp_pct'
end_col = 'sterile_beds_pct'

# Find the index positions of the start and end columns
start_idx = df3_log.columns.get_loc(start_col)
end_idx = df3_log.columns.get_loc(end_col)

# Slice the columns based on their integer positions
exclude_cols = df3_log.columns[start_idx : end_idx + 1].tolist()

In [62]:
# 2. 커스텀 대체 로그값 (이전 코드 활용)
# 'Sales' column is not present in the current dataframe, so removing this specific custom replacement.
# Relying on default_log_value for handling log(0).
default_log_value = 0.0

# Initialize an empty DataFrame to store the log-transformed and selected original columns
df3_log_new = pd.DataFrame()

# Iterate through each column in df3.
# df3 already contains only the numeric columns relevant to the model ('confirmed_per100k' and X_vars).
for col in df3.columns:
    # If the column name is in the exclude_cols list, it means it should NOT be log-transformed.
    if col in exclude_cols:
        # Add the original column to the new DataFrame
        df3_log_new[col] = df3[col]
    else:
        # If not excluded, and it's a numeric column, apply log transformation.
        # All columns in df3 are expected to be numeric, so this check adds robustness.
        if pd.api.types.is_numeric_dtype(df3[col]):
            with np.errstate(divide='ignore'):
                log_series = np.log(df3[col])

            # Replace -inf values (resulting from log(0)) with default_log_value (0.0)
            log_series = log_series.replace(-np.inf, default_log_value)

            # Store the log-transformed column with a 'log_' prefix
            df3_log_new[f'log_{col}'] = log_series
        else:
            # This case should ideally not be reached if df3 contains only numeric model variables.
            print(f"Warning: Column '{col}' in df3 is not numeric and not in exclude_cols. Skipping.")

# Assign the newly created DataFrame to df3_log
df3_log = df3_log_new

print("\n✅ 특정 컬럼 제외 후 로그 변환 완료")


✅ 특정 컬럼 제외 후 로그 변환 완료


In [63]:
df3_log.head(1)

,log_confirmed_per100k,log_docs_total_per100k,log_nurses_total_per100k,log_staff_total_per100k,tertiary_hosp_pct,general_hosp_pct,hospital_pct,clinic_pct,public_health_pct,ward_rooms_pct,icu_rooms_pct,iso_rooms_pct,sterile_rooms_pct,ward_beds_pct,icu_beds_pct,iso_beds_pct,sterile_beds_pct,log_pop_elderly_per100k,log_income_perindiv,log_region_income_perindiv
0,3.997559,5.823958,6.430499,6.865674,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,9.712068,10.20799,10.993698


In [64]:
# Log Y1: 감염률 per100k (로그화) ~ 파생 독립변수 (일부 로그화)
X_log_vars = df3_log.iloc[:,1:].columns.tolist()
formula_X_log = ' + '.join(X_log_vars)
formula_Y1_log = f"{Y_log_var_name} ~ {formula_X_log}"
results_Y1_derive_log = smf.ols(formula=formula_Y1_log, data=df3_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_derive_log.summary())
# R-squared 결정계수 = 설명력
# Adj. R-squared 조정결정계수 = 독립변수의 개수가 다른 모델끼리 비교할 때 사용
# F 검정 결과 = 회귀모델 유의성 검정
# AIC 모델 적합성 비교할 때 사용
# 절편 - 유의함
# 기울기 - 유의함
# Durbin-Watson 2에 가까울수록 오차항이 독립

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.612
Model:                               OLS   Adj. R-squared:                  0.596
Method:                    Least Squares   F-statistic:                     38.54
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           3.41e-70
Time:                           00:35:57   Log-Likelihood:                -736.13
No. Observations:                    408   AIC:                             1506.
Df Residuals:                        391   BIC:                             1574.
Df Model:                             16                                         
Covariance Type:               nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------

# OLS 결과 출력 (감염률)

In [ ]:
file_path2 = '/content/drive/MyDrive/MedicalDA04/Semi Project#1/Analysis Dataset/3. 회귀 분석/'

# 0. OLS summary 확인
# Fetching SimpleTable objects
tables_ori_0 = results_Y1_ori.summary().tables[0]
tables_ori_log_0 = results_Y1_ori_log.summary().tables[0]
tables_derive_0 = results_Y1_derive.summary().tables[0]
tables_derive_log_0 = results_Y1_derive_log.summary().tables[0]

tables_ori_1 = results_Y1_ori.summary().tables[1]
tables_ori_log_1 = results_Y1_ori_log.summary().tables[1]
tables_derive_1 = results_Y1_derive.summary().tables[1]
tables_derive_log_1 = results_Y1_derive_log.summary().tables[1]

tables_ori_2 = results_Y1_ori.summary().tables[2]
tables_ori_log_2 = results_Y1_ori_log.summary().tables[2]
tables_derive_2 = results_Y1_derive.summary().tables[2]
tables_derive_log_2 = results_Y1_derive_log.summary().tables[2]


# 1. 저장할 파일 이름 정의
excel_file_name = f'{Y_var_name}_Basic OLS_Data_Sheets.xlsx'

# 2. 데이터프레임과 저장할 시트 이름을 딕셔너리로 매핑 (핵심)
# Convert SimpleTable objects to pandas DataFrames using as_html and read_html
data_to_save = {
    'OLS_ori_Summary_Top': pd.read_html(tables_ori_0.as_html(), header=None, index_col=0)[0],
    'OLS_ori_log_Summary_Top': pd.read_html(tables_ori_log_0.as_html(), header=None, index_col=0)[0],
    'OLS_derive_Summary_Top': pd.read_html(tables_derive_0.as_html(), header=None, index_col=0)[0],
    'OLS_derive_log_Summary_Top': pd.read_html(tables_derive_log_0.as_html(), header=None, index_col=0)[0],

    'OLS_ori_Coeffs': pd.read_html(tables_ori_1.as_html(), header=0, index_col=0)[0],
    'OLS_ori_log_Coeffs': pd.read_html(tables_ori_log_1.as_html(), header=0, index_col=0)[0],
    'OLS_derive_Coeffs': pd.read_html(tables_derive_1.as_html(), header=0, index_col=0)[0],
    'OLS_derive_log_Coeffs': pd.read_html(tables_derive_log_1.as_html(), header=0, index_col=0)[0],

    'OLS_ori_Tests': pd.read_html(tables_ori_2.as_html(), header=None, index_col=0)[0],
    'OLS_ori_log_Tests': pd.read_html(tables_ori_log_2.as_html(), header=None, index_col=0)[0],
    'OLS_derive_Tests': pd.read_html(tables_derive_2.as_html(), header=None, index_col=0)[0],
    'OLS_derive_log_Tests': pd.read_html(tables_derive_log_2.as_html(), header=None, index_col=0)[0]
}

# 3. pandas.ExcelWriter 객체 생성 및 사용
# with 구문을 사용하면 파일 관리가 자동으로 처리되어 안전합니다.
with pd.ExcelWriter(file_path2 + excel_file_name, engine='xlsxwriter') as writer:
    # 딕셔너리를 순회하며 각 DataFrame을 시트별로 저장
    for sheet_name, dataframe in data_to_save.items():
        # index=True is used because we set index_col=0 when reading the HTML, making the first column the DataFrame index.
        dataframe.to_excel(writer, sheet_name=sheet_name, index=True)

/tmp/ipython-input-3400063122.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_ori_Summary_Top': pd.read_html(tables_ori_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-3400063122.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_ori_log_Summary_Top': pd.read_html(tables_ori_log_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-3400063122.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_derive_Summary_Top': pd.read_html(tables_derive_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-3400063122.py:30: FutureWarning: Passing literal html to 'read_html' is d

# 기본 선형 회귀 모델 (사망률)

In [26]:
df = pd.read_csv(file_path+file_name, header = 0, encoding = ' euc-kr')
df.head(1)

,시도명,시도명 코드,연도,월별,COVID19 확진자 수,COVID19 사망자 수,COVID19 1차 예방 접종자 수,COVID19 2차 예방 접종자 수,COVID19 3차 예방 접종자 수,COVID19 총 예방 접종자 수,총인구(명),총 고령인구(명),상급종합병원,종합병원,병원,의원,보건의료기관,의료인프라 총합,총 의사수(명),총 간호사수(명),총 의료인력 수(명),총 일반입원실 병실수,총 일반입원실 병상수,총 중환자실 병실수,총 중환자실 병상수,총 격리병실 병실수,총 격리병실 병상수,총 무균치료실 병실수,총 무균치료실 병상수,총 의료기관 병실수,총 의료기관 병상수,1인당 개인소득,1인당 지역총소득,감염률_per100k,사망률_per100k,치명률_percent,3차 접종률_percent,총 의사수_per100k,총 간호사수_per100k,총 의료인력 수_per100k,총 상급종합병원_percent,총 종합병원_percent,총 병원_percent,총 의원_percent,총 보건의료기관_percent,총 일반입원실 병실수_percent,총 중환자실 병실수_percent,총 격리병실 병실수_percent,총 무균치료실 병실수_percent,총 일반입원실 병상수_percent,총 중환자실 병상수_percent,총 격리병실 병상수_percent,총 무균치료실 병상수_percent,총 고령인구수_per100k,1인당 개인소득_perIndiv,1인당 지역총소득_perIndiv
0,서울,11,2021,1,5159,147,0,0,0,0,9472127,1564390,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,54.465064,1.551922,2.849389,0.0,338.308386,620.483657,958.792043,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,16515.720281,27119,59498


In [27]:
df1 = df.copy()
columns_eng = [
    "region", "region_code", "year", "month",
    "confirmed", "deaths",
    "vax1", "vax2", "vax3", "vax_total",
    "pop_total", "pop_elderly",
    "tertiary_hosp", "general_hosp", "hospital", "clinic", "public_health",
    "infra_total",
    "docs_total", "nurses_total", "staff_total",
    "ward_rooms", "ward_beds",
    "icu_rooms", "icu_beds",
    "iso_rooms", "iso_beds",
    "sterile_rooms", "sterile_beds",
    "rooms_total", "beds_total",
    "income_indiv", "region_income_indiv",
    "confirmed_per100k", "death_per100k", "crit_pct", "vax3_pct",
    "docs_total_per100k", "nurses_total_per100k", "staff_total_per100k",
    "tertiary_hosp_pct", "general_hosp_pct", "hospital_pct", "clinic_pct", "public_health_pct",
    "ward_rooms_pct", "icu_rooms_pct", "iso_rooms_pct", "sterile_rooms_pct",
    "ward_beds_pct", "icu_beds_pct", "iso_beds_pct", "sterile_beds_pct",
    "pop_elderly_per100k", "income_perindiv", "region_income_perindiv"
]

df1.columns = columns_eng

In [28]:
df1.head(1)

,region,region_code,year,month,confirmed,deaths,vax1,vax2,vax3,vax_total,pop_total,pop_elderly,tertiary_hosp,general_hosp,hospital,clinic,public_health,infra_total,docs_total,nurses_total,staff_total,ward_rooms,ward_beds,icu_rooms,icu_beds,iso_rooms,iso_beds,sterile_rooms,sterile_beds,rooms_total,beds_total,income_indiv,region_income_indiv,confirmed_per100k,death_per100k,crit_pct,vax3_pct,docs_total_per100k,nurses_total_per100k,staff_total_per100k,tertiary_hosp_pct,general_hosp_pct,hospital_pct,clinic_pct,public_health_pct,ward_rooms_pct,icu_rooms_pct,iso_rooms_pct,sterile_rooms_pct,ward_beds_pct,icu_beds_pct,iso_beds_pct,sterile_beds_pct,pop_elderly_per100k,income_perindiv,region_income_perindiv
0,서울,11,2021,1,5159,147,0,0,0,0,9472127,1564390,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,54.465064,1.551922,2.849389,0.0,338.308386,620.483657,958.792043,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,16515.720281,27119,59498


In [29]:
# ============================================================================
# 0 값 확인
# ============================================================================
dependent_vars = df1.columns[4:]

print("\n" + "=" * 100)
print("0 값 확인")
print("=" * 100)

zero_analysis = []

for var in dependent_vars:
    # missing_count = df1[var].isnull().sum()
    zero_count = (df1[var]==0).sum()
    zero_percent = (zero_count / len(df1)) * 100

    zero_analysis.append({
        '변수': var,
        '0값 개수': zero_count,
        '0값 비율(%)': zero_percent,
    })

zero_df = pd.DataFrame(zero_analysis)

print("\n0 값 분석\n")
# print(zero_df.to_string(index=False))
# zero_df.to_excel(file_path + 'zero_analysis.xlsx', sheet_name='zero_analysis', index=True)


0 값 확인

0 값 분석



In [30]:
X_vars = df1.loc[:,'tertiary_hosp':'region_income_indiv'].columns.tolist() + ['pop_elderly']
Y_var_name = 'death_per100k'
Y_var = [Y_var_name]

Y_log_var_name = 'log_death_per100k'

In [31]:
df2 = df1.loc[:, Y_var + X_vars]
df2.head(1)

,death_per100k,tertiary_hosp,general_hosp,hospital,clinic,public_health,infra_total,docs_total,nurses_total,staff_total,ward_rooms,ward_beds,icu_rooms,icu_beds,iso_rooms,iso_beds,sterile_rooms,sterile_beds,rooms_total,beds_total,income_indiv,region_income_indiv,pop_elderly
0,1.551922,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,1564390


In [32]:
# Y1: 감염률 per100k ~ 원시 독립변수
formula_X = ' + '.join(X_vars)
formula_Y1 = f"{Y_var_name} ~ {formula_X}"
results_Y1_ori = smf.ols(formula=formula_Y1, data=df2).fit()

print("### OLS 모델 결과 (Y1: 변화율 그대로) ###")
print(results_Y1_ori.summary())
# R-squared 결정계수 = 설명력
# Adj. R-squared 조정결정계수 = 독립변수의 개수가 다른 모델끼리 비교할 때 사용
# F 검정 결과 = 회귀모델 유의성 검정
# AIC 모델 적합성 비교할 때 사용
# 절편 - 유의함
# 기울기 - 유의함
# Durbin-Watson 2에 가까울수록 오차항이 독립

### OLS 모델 결과 (Y1: 변화율 그대로) ###
                            OLS Regression Results                            
Dep. Variable:          death_per100k   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     6.072
Date:                Thu, 18 Dec 2025   Prob (F-statistic):           4.61e-13
Time:                        00:20:08   Log-Likelihood:                -1108.6
No. Observations:                 408   AIC:                             2255.
Df Residuals:                     389   BIC:                             2331.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
In

In [33]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
df2_log = df2.copy()

# 1. 컬럼별로 -inf(로그 0) 대신 넣고 싶은 '로그 값' 지정
# 주의: 이 값은 로그가 취해진 상태의 값입니다. (예: 0, -1, -5 등)
custom_log_replace_values = {
    'tertiary_hosp': 0,   # 로그 변환 후 -inf라면 0.1으로 대체
    'sterile_rooms': 0,
    'sterile_beds': 0
}

# 기본 대체 로그값 (딕셔너리에 없는 컬럼들이 사용할 값, 로그 1 = 0 이므로 0.0 권장)
default_log_value = 0.0

# 원본 컬럼 리스트 고정
original_columns = df2_log.columns.tolist()

# 2. 순회 및 변환
for col in original_columns:
    # (A) 먼저 로그 변환 수행 (0인 지점은 -inf가 됨)
    # RuntimeWarning을 무시하기 위해 errstate 사용
    with np.errstate(divide='ignore'):
        log_series = np.log(df2_log[col])

    # (B) 해당 컬럼의 대체 로그값 결정
    replacement = custom_log_replace_values.get(col, default_log_value)

    # (C) 결과가 -inf인 것만 찾아서 replacement로 치환
    # np.isinf()를 사용하여 음의 무한대(-inf)를 체크합니다.
    log_series = log_series.replace(-np.inf, replacement)

    # (D) 새로운 컬럼으로 저장
    df2_log[f'log_{col}'] = log_series

# 3. 'log_'가 포함된 컬럼 이름들만 필터링하여 최종 추출
columns_log = [col for col in df2_log.columns if 'log_' in col.lower()]
df2_log = df2_log[columns_log]

print("✅ 로그 변환 후 -inf 값 사후 처리 완료")

✅ 로그 변환 후 -inf 값 사후 처리 완료


In [34]:
df2_log.head(1)

,log_death_per100k,log_tertiary_hosp,log_general_hosp,log_hospital,log_clinic,log_public_health,log_infra_total,log_docs_total,log_nurses_total,log_staff_total,log_ward_rooms,log_ward_beds,log_icu_rooms,log_icu_beds,log_iso_rooms,log_iso_beds,log_sterile_rooms,log_sterile_beds,log_rooms_total,log_beds_total,log_income_indiv,log_region_income_indiv,log_pop_elderly
0,0.439494,2.639057,3.73767,5.402677,9.125762,3.496508,9.159047,10.374896,10.981438,11.416613,10.140258,11.336499,5.521461,7.886081,6.45677,7.186901,4.905275,5.370638,10.179793,11.385353,10.20799,10.993698,14.263007


In [35]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
X_log_vars = columns_log[1:]
formula_X_log = ' + '.join(X_log_vars)
formula_Y1_log = f"{Y_log_var_name} ~ {formula_X_log}"
results_Y1_ori_log = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log.summary())
# R-squared 결정계수 = 설명력
# Adj. R-squared 조정결정계수 = 독립변수의 개수가 다른 모델끼리 비교할 때 사용
# F 검정 결과 = 회귀모델 유의성 검정
# AIC 모델 적합성 비교할 때 사용
# 절편 - 유의함
# 기울기 - 유의함
# Durbin-Watson 2에 가까울수록 오차항이 독립

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                            OLS Regression Results                            
Dep. Variable:      log_death_per100k   R-squared:                       0.478
Model:                            OLS   Adj. R-squared:                  0.448
Method:                 Least Squares   F-statistic:                     16.02
Date:                Thu, 18 Dec 2025   Prob (F-statistic):           7.55e-42
Time:                        00:20:08   Log-Likelihood:                -608.81
No. Observations:                 408   AIC:                             1264.
Df Residuals:                     385   BIC:                             1356.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [36]:
df1.head(1)

,region,region_code,year,month,confirmed,deaths,vax1,vax2,vax3,vax_total,pop_total,pop_elderly,tertiary_hosp,general_hosp,hospital,clinic,public_health,infra_total,docs_total,nurses_total,staff_total,ward_rooms,ward_beds,icu_rooms,icu_beds,iso_rooms,iso_beds,sterile_rooms,sterile_beds,rooms_total,beds_total,income_indiv,region_income_indiv,confirmed_per100k,death_per100k,crit_pct,vax3_pct,docs_total_per100k,nurses_total_per100k,staff_total_per100k,tertiary_hosp_pct,general_hosp_pct,hospital_pct,clinic_pct,public_health_pct,ward_rooms_pct,icu_rooms_pct,iso_rooms_pct,sterile_rooms_pct,ward_beds_pct,icu_beds_pct,iso_beds_pct,sterile_beds_pct,pop_elderly_per100k,income_perindiv,region_income_perindiv
0,서울,11,2021,1,5159,147,0,0,0,0,9472127,1564390,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,54.465064,1.551922,2.849389,0.0,338.308386,620.483657,958.792043,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,16515.720281,27119,59498


In [37]:
df3 = df1.copy()
X_vars = df1.loc[:,'docs_total_per100k':'region_income_perindiv'].columns.tolist()
Y_var = [Y_var_name]

In [38]:
df3 = df3.loc[:, Y_var + X_vars]
df3.head(1)

,death_per100k,docs_total_per100k,nurses_total_per100k,staff_total_per100k,tertiary_hosp_pct,general_hosp_pct,hospital_pct,clinic_pct,public_health_pct,ward_rooms_pct,icu_rooms_pct,iso_rooms_pct,sterile_rooms_pct,ward_beds_pct,icu_beds_pct,iso_beds_pct,sterile_beds_pct,pop_elderly_per100k,income_perindiv,region_income_perindiv
0,1.551922,338.308386,620.483657,958.792043,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,16515.720281,27119,59498


In [39]:
# Y1: 감염률 per100k ~ 파생 독립변수
formula_X = ' + '.join(X_vars)
formula_Y1 = f"{Y_var_name} ~ {formula_X}"
results_Y1_derive = smf.ols(formula=formula_Y1, data=df3).fit()

print("### OLS 모델 결과 (Y1: 변화율 그대로) ###")
print(results_Y1_derive.summary())
# R-squared 결정계수 = 설명력
# Adj. R-squared 조정결정계수 = 독립변수의 개수가 다른 모델끼리 비교할 때 사용
# F 검정 결과 = 회귀모델 유의성 검정
# AIC 모델 적합성 비교할 때 사용
# 절편 - 유의함
# 기울기 - 유의함
# Durbin-Watson 2에 가까울수록 오차항이 독립

### OLS 모델 결과 (Y1: 변화율 그대로) ###
                            OLS Regression Results                            
Dep. Variable:          death_per100k   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     5.836
Date:                Thu, 18 Dec 2025   Prob (F-statistic):           6.17e-11
Time:                        00:20:09   Log-Likelihood:                -1118.0
No. Observations:                 408   AIC:                             2268.
Df Residuals:                     392   BIC:                             2332.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

In [40]:
# Log Y1: 감염률 per100k (로그화) ~ 파생 독립변수 (일부 로그화)
df3_log = df1.copy()

# 1. 제외하고 싶은 컬럼 리스트 정의
# Correct approach to select a range of columns
start_col = 'tertiary_hosp_pct'
end_col = 'sterile_beds_pct'

# Find the index positions of the start and end columns
start_idx = df3_log.columns.get_loc(start_col)
end_idx = df3_log.columns.get_loc(end_col)

# Slice the columns based on their integer positions
exclude_cols = df3_log.columns[start_idx : end_idx + 1].tolist()

In [41]:
# 2. 커스텀 대체 로그값 (이전 코드 활용)
# 'Sales' column is not present in the current dataframe, so removing this specific custom replacement.
# Relying on default_log_value for handling log(0).
default_log_value = 0.0

# Initialize an empty DataFrame to store the log-transformed and selected original columns
df3_log_new = pd.DataFrame()

# Iterate through each column in df3.
# df3 already contains only the numeric columns relevant to the model ('confirmed_per100k' and X_vars).
for col in df3.columns:
    # If the column name is in the exclude_cols list, it means it should NOT be log-transformed.
    if col in exclude_cols:
        # Add the original column to the new DataFrame
        df3_log_new[col] = df3[col]
    else:
        # If not excluded, and it's a numeric column, apply log transformation.
        # All columns in df3 are expected to be numeric, so this check adds robustness.
        if pd.api.types.is_numeric_dtype(df3[col]):
            with np.errstate(divide='ignore'):
                log_series = np.log(df3[col])

            # Replace -inf values (resulting from log(0)) with default_log_value (0.0)
            log_series = log_series.replace(-np.inf, default_log_value)

            # Store the log-transformed column with a 'log_' prefix
            df3_log_new[f'log_{col}'] = log_series
        else:
            # This case should ideally not be reached if df3 contains only numeric model variables.
            print(f"Warning: Column '{col}' in df3 is not numeric and not in exclude_cols. Skipping.")

# Assign the newly created DataFrame to df3_log
df3_log = df3_log_new

print("\n✅ 특정 컬럼 제외 후 로그 변환 완료")


✅ 특정 컬럼 제외 후 로그 변환 완료


In [42]:
df3_log.head(1)

,log_death_per100k,log_docs_total_per100k,log_nurses_total_per100k,log_staff_total_per100k,tertiary_hosp_pct,general_hosp_pct,hospital_pct,clinic_pct,public_health_pct,ward_rooms_pct,icu_rooms_pct,iso_rooms_pct,sterile_rooms_pct,ward_beds_pct,icu_beds_pct,iso_beds_pct,sterile_beds_pct,log_pop_elderly_per100k,log_income_perindiv,log_region_income_perindiv
0,0.439494,5.823958,6.430499,6.865674,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,9.712068,10.20799,10.993698


In [43]:
# Log Y1: 감염률 per100k (로그화) ~ 파생 독립변수 (일부 로그화)
X_log_vars = df3_log.iloc[:,1:].columns.tolist()
formula_X_log = ' + '.join(X_log_vars)
formula_Y1_log = f"{Y_log_var_name} ~ {formula_X_log}"
results_Y1_derive_log = smf.ols(formula=formula_Y1_log, data=df3_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_derive_log.summary())
# R-squared 결정계수 = 설명력
# Adj. R-squared 조정결정계수 = 독립변수의 개수가 다른 모델끼리 비교할 때 사용
# F 검정 결과 = 회귀모델 유의성 검정
# AIC 모델 적합성 비교할 때 사용
# 절편 - 유의함
# 기울기 - 유의함
# Durbin-Watson 2에 가까울수록 오차항이 독립

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                            OLS Regression Results                            
Dep. Variable:      log_death_per100k   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     13.62
Date:                Thu, 18 Dec 2025   Prob (F-statistic):           4.96e-29
Time:                        00:20:09   Log-Likelihood:                -651.07
No. Observations:                 408   AIC:                             1336.
Df Residuals:                     391   BIC:                             1404.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

# OLS 결과 출력 (사망률)

In [ ]:
file_path2 = '/content/drive/MyDrive/MedicalDA04/Semi Project#1/Analysis Dataset/3. 회귀 분석/'

# 0. OLS summary 확인
# Fetching SimpleTable objects
tables_ori_0 = results_Y1_ori.summary().tables[0]
tables_ori_log_0 = results_Y1_ori_log.summary().tables[0]
tables_derive_0 = results_Y1_derive.summary().tables[0]
tables_derive_log_0 = results_Y1_derive_log.summary().tables[0]

tables_ori_1 = results_Y1_ori.summary().tables[1]
tables_ori_log_1 = results_Y1_ori_log.summary().tables[1]
tables_derive_1 = results_Y1_derive.summary().tables[1]
tables_derive_log_1 = results_Y1_derive_log.summary().tables[1]

tables_ori_2 = results_Y1_ori.summary().tables[2]
tables_ori_log_2 = results_Y1_ori_log.summary().tables[2]
tables_derive_2 = results_Y1_derive.summary().tables[2]
tables_derive_log_2 = results_Y1_derive_log.summary().tables[2]


# 1. 저장할 파일 이름 정의
excel_file_name = f'{Y_var_name}_Basic OLS_Data_Sheets.xlsx'

# 2. 데이터프레임과 저장할 시트 이름을 딕셔너리로 매핑 (핵심)
# Convert SimpleTable objects to pandas DataFrames using as_html and read_html
data_to_save = {
    'OLS_ori_Summary_Top': pd.read_html(tables_ori_0.as_html(), header=None, index_col=0)[0],
    'OLS_ori_log_Summary_Top': pd.read_html(tables_ori_log_0.as_html(), header=None, index_col=0)[0],
    'OLS_derive_Summary_Top': pd.read_html(tables_derive_0.as_html(), header=None, index_col=0)[0],
    'OLS_derive_log_Summary_Top': pd.read_html(tables_derive_log_0.as_html(), header=None, index_col=0)[0],

    'OLS_ori_Coeffs': pd.read_html(tables_ori_1.as_html(), header=0, index_col=0)[0],
    'OLS_ori_log_Coeffs': pd.read_html(tables_ori_log_1.as_html(), header=0, index_col=0)[0],
    'OLS_derive_Coeffs': pd.read_html(tables_derive_1.as_html(), header=0, index_col=0)[0],
    'OLS_derive_log_Coeffs': pd.read_html(tables_derive_log_1.as_html(), header=0, index_col=0)[0],

    'OLS_ori_Tests': pd.read_html(tables_ori_2.as_html(), header=None, index_col=0)[0],
    'OLS_ori_log_Tests': pd.read_html(tables_ori_log_2.as_html(), header=None, index_col=0)[0],
    'OLS_derive_Tests': pd.read_html(tables_derive_2.as_html(), header=None, index_col=0)[0],
    'OLS_derive_log_Tests': pd.read_html(tables_derive_log_2.as_html(), header=None, index_col=0)[0]
}

# 3. pandas.ExcelWriter 객체 생성 및 사용
# with 구문을 사용하면 파일 관리가 자동으로 처리되어 안전합니다.
with pd.ExcelWriter(file_path2 + excel_file_name, engine='xlsxwriter') as writer:
    # 딕셔너리를 순회하며 각 DataFrame을 시트별로 저장
    for sheet_name, dataframe in data_to_save.items():
        # index=True is used because we set index_col=0 when reading the HTML, making the first column the DataFrame index.
        dataframe.to_excel(writer, sheet_name=sheet_name, index=True)

/tmp/ipython-input-3400063122.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_ori_Summary_Top': pd.read_html(tables_ori_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-3400063122.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_ori_log_Summary_Top': pd.read_html(tables_ori_log_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-3400063122.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_derive_Summary_Top': pd.read_html(tables_derive_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-3400063122.py:30: FutureWarning: Passing literal html to 'read_html' is d

# 1차 계층 통계 모델 선정 과정

In [126]:
df = pd.read_csv(file_path+file_name, header = 0, encoding = ' euc-kr')
df.head(1)

,시도명,시도명 코드,연도,월별,COVID19 확진자 수,COVID19 사망자 수,COVID19 1차 예방 접종자 수,COVID19 2차 예방 접종자 수,COVID19 3차 예방 접종자 수,COVID19 총 예방 접종자 수,총인구(명),총 고령인구(명),상급종합병원,종합병원,병원,의원,보건의료기관,의료인프라 총합,총 의사수(명),총 간호사수(명),총 의료인력 수(명),총 일반입원실 병실수,총 일반입원실 병상수,총 중환자실 병실수,총 중환자실 병상수,총 격리병실 병실수,총 격리병실 병상수,총 무균치료실 병실수,총 무균치료실 병상수,총 의료기관 병실수,총 의료기관 병상수,1인당 개인소득,1인당 지역총소득,감염률_per100k,사망률_per100k,치명률_percent,3차 접종률_percent,총 의사수_per100k,총 간호사수_per100k,총 의료인력 수_per100k,총 상급종합병원_percent,총 종합병원_percent,총 병원_percent,총 의원_percent,총 보건의료기관_percent,총 일반입원실 병실수_percent,총 중환자실 병실수_percent,총 격리병실 병실수_percent,총 무균치료실 병실수_percent,총 일반입원실 병상수_percent,총 중환자실 병상수_percent,총 격리병실 병상수_percent,총 무균치료실 병상수_percent,총 고령인구수_per100k,1인당 개인소득_perIndiv,1인당 지역총소득_perIndiv
0,서울,11,2021,1,5159,147,0,0,0,0,9472127,1564390,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,54.465064,1.551922,2.849389,0.0,338.308386,620.483657,958.792043,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,16515.720281,27119,59498


In [127]:
df1 = df.copy()
columns_eng = [
    "region", "region_code", "year", "month",
    "confirmed", "deaths",
    "vax1", "vax2", "vax3", "vax_total",
    "pop_total", "pop_elderly",
    "tertiary_hosp", "general_hosp", "hospital", "clinic", "public_health",
    "infra_total",
    "docs_total", "nurses_total", "staff_total",
    "ward_rooms", "ward_beds",
    "icu_rooms", "icu_beds",
    "iso_rooms", "iso_beds",
    "sterile_rooms", "sterile_beds",
    "rooms_total", "beds_total",
    "income_indiv", "region_income_indiv",
    "confirmed_per100k", "death_per100k", "crit_pct", "vax3_pct",
    "docs_total_per100k", "nurses_total_per100k", "staff_total_per100k",
    "tertiary_hosp_pct", "general_hosp_pct", "hospital_pct", "clinic_pct", "public_health_pct",
    "ward_rooms_pct", "icu_rooms_pct", "iso_rooms_pct", "sterile_rooms_pct",
    "ward_beds_pct", "icu_beds_pct", "iso_beds_pct", "sterile_beds_pct",
    "pop_elderly_per100k", "income_perindiv", "region_income_perindiv"
]

df1.columns = columns_eng

In [128]:
X_vars = df1.loc[:,'tertiary_hosp':'region_income_indiv'].columns.tolist() + ['pop_elderly']
Y_var_name1 = 'confirmed_per100k'
Y_var_name2 = 'death_per100k'
Y_var = [Y_var_name1] + [Y_var_name2]

Y_log_var_name1 = 'log_confirmed_per100k'
Y_log_var_name2 = 'log_death_per100k'

In [129]:
df2 = df1.loc[:, Y_var + X_vars]
df2.head(1)

,confirmed_per100k,death_per100k,tertiary_hosp,general_hosp,hospital,clinic,public_health,infra_total,docs_total,nurses_total,staff_total,ward_rooms,ward_beds,icu_rooms,icu_beds,iso_rooms,iso_beds,sterile_rooms,sterile_beds,rooms_total,beds_total,income_indiv,region_income_indiv,pop_elderly
0,54.465064,1.551922,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,1564390


In [130]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화) - 기본 OLS 탐색 결과 참고하여 선정
df2_log = df2.copy()

# 1. 컬럼별로 -inf(로그 0) 대신 넣고 싶은 '로그 값' 지정
# 주의: 이 값은 로그가 취해진 상태의 값입니다. (예: 0, -1, -5 등)
custom_log_replace_values = {
    'tertiary_hosp': 0,   # 로그 변환 후 -inf라면 0으로 대체
    'sterile_rooms': 0,
    'sterile_beds': 0
}

# 기본 대체 로그값 (딕셔너리에 없는 컬럼들이 사용할 값, 로그 1 = 0 이므로 0.0 권장)
default_log_value = 0.0

# 원본 컬럼 리스트 고정
original_columns = df2_log.columns.tolist()

# 2. 순회 및 변환
for col in original_columns:
    # (A) 먼저 로그 변환 수행 (0인 지점은 -inf가 됨)
    # RuntimeWarning을 무시하기 위해 errstate 사용
    with np.errstate(divide='ignore'):
        log_series = np.log(df2_log[col])

    # (B) 해당 컬럼의 대체 로그값 결정
    replacement = custom_log_replace_values.get(col, default_log_value)

    # (C) 결과가 -inf인 것만 찾아서 replacement로 치환
    # np.isinf()를 사용하여 음의 무한대(-inf)를 체크합니다.
    log_series = log_series.replace(-np.inf, replacement)

    # (D) 새로운 컬럼으로 저장
    df2_log[f'log_{col}'] = log_series

# 3. 'log_'가 포함된 컬럼 이름들만 필터링하여 최종 추출
columns_log = [col for col in df2_log.columns if 'log_' in col.lower()]
df2_log = df2_log[columns_log]

print("✅ 로그 변환 후 -inf 값 사후 처리 완료")

✅ 로그 변환 후 -inf 값 사후 처리 완료


In [131]:
df2_log.head(2)

,log_confirmed_per100k,log_death_per100k,log_tertiary_hosp,log_general_hosp,log_hospital,log_clinic,log_public_health,log_infra_total,log_docs_total,log_nurses_total,log_staff_total,log_ward_rooms,log_ward_beds,log_icu_rooms,log_icu_beds,log_iso_rooms,log_iso_beds,log_sterile_rooms,log_sterile_beds,log_rooms_total,log_beds_total,log_income_indiv,log_region_income_indiv,log_pop_elderly
0,3.997559,0.439494,2.639057,3.73767,5.402677,9.125762,3.496508,9.159047,10.374896,10.981438,11.416613,10.140258,11.336499,5.521461,7.886081,6.45677,7.186901,4.905275,5.370638,10.179793,11.385353,10.20799,10.993698,14.263007
1,3.762669,-0.525587,2.639057,3.73767,5.402677,9.125762,3.496508,9.159047,10.374896,10.981438,11.416613,10.140258,11.336499,5.521461,7.886081,6.45677,7.186901,4.905275,5.370638,10.179793,11.385353,10.20799,10.993698,14.263007


In [132]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Medical institutes)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 전체 컬럼에서 included_cols를 포함한 인덱스 반환
target_words = ['hosp', 'hospital', 'clinic', 'health', 'infra']
included_cols = [col for col in X_log_vars if any(w in col.lower() for w in target_words)]

formula_X_log = ' + '.join(included_cols)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod1 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod1.summary())

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.013
Model:                               OLS   Adj. R-squared:                 -0.002
Method:                    Least Squares   F-statistic:                    0.8716
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):              0.516
Time:                           06:08:32   Log-Likelihood:                -926.62
No. Observations:                    408   AIC:                             1867.
Df Residuals:                        401   BIC:                             1895.
Df Model:                              6                                         
Covariance Type:               nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [133]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Medical institutes + )
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 전체 컬럼에서 included_cols를 포함한 인덱스 반환
target_words = ['hosp', 'hospital', 'clinic', 'health', 'infra',
                'docs', 'nurses', 'staff']
included_cols = [col for col in X_log_vars if any(w in col.lower() for w in target_words)]

formula_X_log = ' + '.join(included_cols)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod2 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod2.summary())

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.069
Model:                               OLS   Adj. R-squared:                  0.048
Method:                    Least Squares   F-statistic:                     3.275
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           0.000733
Time:                           06:08:32   Log-Likelihood:                -914.69
No. Observations:                    408   AIC:                             1849.
Df Residuals:                        398   BIC:                             1889.
Df Model:                              9                                         
Covariance Type:               nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [134]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Medical institutes + )
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 전체 컬럼에서 included_cols를 포함한 인덱스 반환
target_words = ['hosp', 'hospital', 'clinic', 'health', 'infra',
                'docs', 'nurses', 'staff', 'beds']
included_cols = [col for col in X_log_vars if any(w in col.lower() for w in target_words)]

formula_X_log = ' + '.join(included_cols)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod3 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod3.summary())

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.139
Model:                               OLS   Adj. R-squared:                  0.109
Method:                    Least Squares   F-statistic:                     4.549
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           1.21e-07
Time:                           06:08:32   Log-Likelihood:                -898.62
No. Observations:                    408   AIC:                             1827.
Df Residuals:                        393   BIC:                             1887.
Df Model:                             14                                         
Covariance Type:               nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [135]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Medical institutes + )
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 전체 컬럼에서 included_cols를 포함한 인덱스 반환
target_words = ['hosp', 'hospital', 'clinic', 'health', 'infra',
                'docs', 'nurses', 'staff', 'beds', 'income']
included_cols = [col for col in X_log_vars if any(w in col.lower() for w in target_words)]

formula_X_log = ' + '.join(included_cols)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod4 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod4.summary())

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.718
Model:                               OLS   Adj. R-squared:                  0.706
Method:                    Least Squares   F-statistic:                     62.19
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           8.95e-97
Time:                           06:08:33   Log-Likelihood:                -671.11
No. Observations:                    408   AIC:                             1376.
Df Residuals:                        391   BIC:                             1444.
Df Model:                             16                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [136]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Medical institutes + )
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 전체 컬럼에서 included_cols를 포함한 인덱스 반환
target_words = ['hosp', 'hospital', 'clinic', 'health', 'infra',
                'docs', 'nurses', 'staff', 'beds', 'income', 'elderly']
included_cols = [col for col in X_log_vars if any(w in col.lower() for w in target_words)]

formula_X_log = ' + '.join(included_cols)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod5 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod5.summary())

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.718
Model:                               OLS   Adj. R-squared:                  0.706
Method:                    Least Squares   F-statistic:                     58.53
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           4.98e-96
Time:                           06:08:33   Log-Likelihood:                -670.72
No. Observations:                    408   AIC:                             1377.
Df Residuals:                        390   BIC:                             1450.
Df Model:                             17                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

# 1차 Hierachical Model 결과 출력

In [137]:
file_path2 = '/content/drive/MyDrive/MedicalDA04/Semi Project#1/Analysis Dataset/3. 회귀 분석/'

# 0. OLS summary 확인
# Fetching SimpleTable objects
# Model 1 ~ 6
table_num0 = 0
tables_ori_log_mod1_0 = results_Y1_ori_log_mod1.summary().tables[table_num0]
tables_ori_log_mod2_0 = results_Y1_ori_log_mod2.summary().tables[table_num0]
tables_ori_log_mod3_0 = results_Y1_ori_log_mod3.summary().tables[table_num0]
tables_ori_log_mod4_0 = results_Y1_ori_log_mod4.summary().tables[table_num0]
tables_ori_log_mod5_0 = results_Y1_ori_log_mod5.summary().tables[table_num0]

table_num1 = 1
tables_ori_log_mod1_1 = results_Y1_ori_log_mod1.summary().tables[table_num1]
tables_ori_log_mod2_1 = results_Y1_ori_log_mod2.summary().tables[table_num1]
tables_ori_log_mod3_1 = results_Y1_ori_log_mod3.summary().tables[table_num1]
tables_ori_log_mod4_1 = results_Y1_ori_log_mod4.summary().tables[table_num1]
tables_ori_log_mod5_1 = results_Y1_ori_log_mod5.summary().tables[table_num1]

table_num2 = 2
tables_ori_log_mod1_2 = results_Y1_ori_log_mod1.summary().tables[table_num2]
tables_ori_log_mod2_2 = results_Y1_ori_log_mod2.summary().tables[table_num2]
tables_ori_log_mod3_2 = results_Y1_ori_log_mod3.summary().tables[table_num2]
tables_ori_log_mod4_2 = results_Y1_ori_log_mod4.summary().tables[table_num2]
tables_ori_log_mod5_2 = results_Y1_ori_log_mod5.summary().tables[table_num2]

# 1. 저장할 파일 이름 정의
excel_file_name = f'{Y_var_name1}_Hierachical#1 OLS_Data_Sheets.xlsx'

# 2. 데이터프레임과 저장할 시트 이름을 딕셔너리로 매핑 (핵심)
# Convert SimpleTable objects to pandas DataFrames using as_html and read_html
data_to_save = {
    'OLS_log_mod1_Top': pd.read_html(tables_ori_log_mod1_0.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod2_Top': pd.read_html(tables_ori_log_mod2_0.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod3_Top': pd.read_html(tables_ori_log_mod3_0.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod4_Top': pd.read_html(tables_ori_log_mod4_0.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod5_Top': pd.read_html(tables_ori_log_mod5_0.as_html(), header=None, index_col=0)[0],

    'OLS_log_mod1_Tests': pd.read_html(tables_ori_log_mod1_2.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod2_Tests': pd.read_html(tables_ori_log_mod2_2.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod3_Tests': pd.read_html(tables_ori_log_mod3_2.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod4_Tests': pd.read_html(tables_ori_log_mod4_2.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod5_Tests': pd.read_html(tables_ori_log_mod5_2.as_html(), header=None, index_col=0)[0],

    'OLS_log_mod1_Coeffs': pd.read_html(tables_ori_log_mod1_1.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod2_Coeffs': pd.read_html(tables_ori_log_mod2_1.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod3_Coeffs': pd.read_html(tables_ori_log_mod3_1.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod4_Coeffs': pd.read_html(tables_ori_log_mod4_1.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod5_Coeffs': pd.read_html(tables_ori_log_mod5_1.as_html(), header=None, index_col=0)[0],
}

# 3. pandas.ExcelWriter 객체 생성 및 사용
# with 구문을 사용하면 파일 관리가 자동으로 처리되어 안전합니다.
with pd.ExcelWriter(file_path2 + excel_file_name, engine='xlsxwriter') as writer:
    # 딕셔너리를 순회하며 각 DataFrame을 시트별로 저장
    for sheet_name, dataframe in data_to_save.items():
        # index=True is used because we set index_col=0 when reading the HTML, making the first column the DataFrame index.
        dataframe.to_excel(writer, sheet_name=sheet_name, index=True)

/tmp/ipython-input-1686378925.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_log_mod1_Top': pd.read_html(tables_ori_log_mod1_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-1686378925.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_log_mod2_Top': pd.read_html(tables_ori_log_mod2_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-1686378925.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_log_mod3_Top': pd.read_html(tables_ori_log_mod3_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-1686378925.py:36: FutureWarning: Passing literal html to 'read_html' 

# 1-1차 계층 통계 모델 선정 과정

In [145]:
df = pd.read_csv(file_path+file_name, header = 0, encoding = ' euc-kr')
df.head(1)

,시도명,시도명 코드,연도,월별,COVID19 확진자 수,COVID19 사망자 수,COVID19 1차 예방 접종자 수,COVID19 2차 예방 접종자 수,COVID19 3차 예방 접종자 수,COVID19 총 예방 접종자 수,총인구(명),총 고령인구(명),상급종합병원,종합병원,병원,의원,보건의료기관,의료인프라 총합,총 의사수(명),총 간호사수(명),총 의료인력 수(명),총 일반입원실 병실수,총 일반입원실 병상수,총 중환자실 병실수,총 중환자실 병상수,총 격리병실 병실수,총 격리병실 병상수,총 무균치료실 병실수,총 무균치료실 병상수,총 의료기관 병실수,총 의료기관 병상수,1인당 개인소득,1인당 지역총소득,감염률_per100k,사망률_per100k,치명률_percent,3차 접종률_percent,총 의사수_per100k,총 간호사수_per100k,총 의료인력 수_per100k,총 상급종합병원_percent,총 종합병원_percent,총 병원_percent,총 의원_percent,총 보건의료기관_percent,총 일반입원실 병실수_percent,총 중환자실 병실수_percent,총 격리병실 병실수_percent,총 무균치료실 병실수_percent,총 일반입원실 병상수_percent,총 중환자실 병상수_percent,총 격리병실 병상수_percent,총 무균치료실 병상수_percent,총 고령인구수_per100k,1인당 개인소득_perIndiv,1인당 지역총소득_perIndiv
0,서울,11,2021,1,5159,147,0,0,0,0,9472127,1564390,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,54.465064,1.551922,2.849389,0.0,338.308386,620.483657,958.792043,0.147368,0.442105,2.336842,96.726316,0.347368,96.123649,0.948227,2.416082,0.512042,95.231928,3.021937,1.50188,0.244254,16515.720281,27119,59498


In [146]:
df1 = df.copy()
columns_eng = [
    "region", "region_code", "year", "month",
    "confirmed", "deaths",
    "vax1", "vax2", "vax3", "vax_total",
    "pop_total", "pop_elderly",
    "tertiary_hosp", "general_hosp", "hospital", "clinic", "public_health",
    "infra_total",
    "docs_total", "nurses_total", "staff_total",
    "ward_rooms", "ward_beds",
    "icu_rooms", "icu_beds",
    "iso_rooms", "iso_beds",
    "sterile_rooms", "sterile_beds",
    "rooms_total", "beds_total",
    "income_indiv", "region_income_indiv",
    "confirmed_per100k", "death_per100k", "crit_pct", "vax3_pct",
    "docs_total_per100k", "nurses_total_per100k", "staff_total_per100k",
    "tertiary_hosp_pct", "general_hosp_pct", "hospital_pct", "clinic_pct", "public_health_pct",
    "ward_rooms_pct", "icu_rooms_pct", "iso_rooms_pct", "sterile_rooms_pct",
    "ward_beds_pct", "icu_beds_pct", "iso_beds_pct", "sterile_beds_pct",
    "pop_elderly_per100k", "income_perindiv", "region_income_perindiv"
]

df1.columns = columns_eng

In [147]:
X_vars = df1.loc[:,'tertiary_hosp':'region_income_indiv'].columns.tolist() + ['pop_elderly']
Y_var_name1 = 'confirmed_per100k'
Y_var_name2 = 'death_per100k'
Y_var = [Y_var_name1] + [Y_var_name2]

Y_log_var_name1 = 'log_confirmed_per100k'
Y_log_var_name2 = 'log_death_per100k'

In [148]:
df2 = df1.loc[:, Y_var + X_vars]
df2.head(1)

,confirmed_per100k,death_per100k,tertiary_hosp,general_hosp,hospital,clinic,public_health,infra_total,docs_total,nurses_total,staff_total,ward_rooms,ward_beds,icu_rooms,icu_beds,iso_rooms,iso_beds,sterile_rooms,sterile_beds,rooms_total,beds_total,income_indiv,region_income_indiv,pop_elderly
0,54.465064,1.551922,14,42,222,9189,33,9500,32045,58773,90818,25343,83826,250,2660,637,1322,135,215,26365,88023,27119,59498,1564390


In [149]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화) - 기본 OLS 탐색 결과 참고하여 선정
df2_log = df2.copy()

# 1. 컬럼별로 -inf(로그 0) 대신 넣고 싶은 '로그 값' 지정
# 주의: 이 값은 로그가 취해진 상태의 값입니다. (예: 0, -1, -5 등)
custom_log_replace_values = {
    'tertiary_hosp': 0,   # 로그 변환 후 -inf라면 0으로 대체
    'sterile_rooms': 0,
    'sterile_beds': 0
}

# 기본 대체 로그값 (딕셔너리에 없는 컬럼들이 사용할 값, 로그 1 = 0 이므로 0.0 권장)
default_log_value = 0.0

# 원본 컬럼 리스트 고정
original_columns = df2_log.columns.tolist()

# 2. 순회 및 변환
for col in original_columns:
    # (A) 먼저 로그 변환 수행 (0인 지점은 -inf가 됨)
    # RuntimeWarning을 무시하기 위해 errstate 사용
    with np.errstate(divide='ignore'):
        log_series = np.log(df2_log[col])

    # (B) 해당 컬럼의 대체 로그값 결정
    replacement = custom_log_replace_values.get(col, default_log_value)

    # (C) 결과가 -inf인 것만 찾아서 replacement로 치환
    # np.isinf()를 사용하여 음의 무한대(-inf)를 체크합니다.
    log_series = log_series.replace(-np.inf, replacement)

    # (D) 새로운 컬럼으로 저장
    df2_log[f'log_{col}'] = log_series

# 3. 'log_'가 포함된 컬럼 이름들만 필터링하여 최종 추출
columns_log = [col for col in df2_log.columns if 'log_' in col.lower()]
df2_log = df2_log[columns_log]

print("✅ 로그 변환 후 -inf 값 사후 처리 완료")

✅ 로그 변환 후 -inf 값 사후 처리 완료


In [150]:
df2_log.head(2)

,log_confirmed_per100k,log_death_per100k,log_tertiary_hosp,log_general_hosp,log_hospital,log_clinic,log_public_health,log_infra_total,log_docs_total,log_nurses_total,log_staff_total,log_ward_rooms,log_ward_beds,log_icu_rooms,log_icu_beds,log_iso_rooms,log_iso_beds,log_sterile_rooms,log_sterile_beds,log_rooms_total,log_beds_total,log_income_indiv,log_region_income_indiv,log_pop_elderly
0,3.997559,0.439494,2.639057,3.73767,5.402677,9.125762,3.496508,9.159047,10.374896,10.981438,11.416613,10.140258,11.336499,5.521461,7.886081,6.45677,7.186901,4.905275,5.370638,10.179793,11.385353,10.20799,10.993698,14.263007
1,3.762669,-0.525587,2.639057,3.73767,5.402677,9.125762,3.496508,9.159047,10.374896,10.981438,11.416613,10.140258,11.336499,5.521461,7.886081,6.45677,7.186901,4.905275,5.370638,10.179793,11.385353,10.20799,10.993698,14.263007


In [164]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Income)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 1. 단어 순서대로 컬럼을 찾아서 리스트 생성
target_words = ['income']

ordered_cols = [
    col for word in target_words
    for col in X_log_vars
    if word in col.lower()
]
# included_cols = [col for col in X_log_vars if any(w in col.lower() for w in target_words)]

# 2. 중복 제거 (한 컬럼에 여러 단어가 포함된 경우 대비)
seen = set()
included_cols = [c for c in ordered_cols if not (c in seen or seen.add(c))]

formula_X_log = ' + '.join(included_cols)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod1 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod1.summary())

# ------------------------------------------------------------------------------
# VIF 확인
# 1. 분석에 사용할 독립변수(X) 리스트 정의 (이전에 추출한 컬럼 리스트 활용)
# 예: filter_X_log_vars를 사용한다고 가정
X_data = df2_log[included_cols]

# 2. VIF 계산을 위해 상수항(intercept) 추가
# statsmodels의 VIF 공식은 상수항이 포함된 것을 전제로 합니다.
X_data_with_const = sm.add_constant(X_data)

# 3. VIF 계산
vif_Y1_mod1 = pd.DataFrame()
vif_Y1_mod1["feature"] = X_data_with_const.columns
vif_Y1_mod1["VIF"] = [variance_inflation_factor(X_data_with_const.values, i)
                   for i in range(len(X_data_with_const.columns))]

# 4. 결과 출력 (보통 상수항인 'const'는 제외하고 확인합니다)
print(vif_Y1_mod1[vif_Y1_mod1['feature'] != 'const'])

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.179
Model:                               OLS   Adj. R-squared:                  0.175
Method:                    Least Squares   F-statistic:                     44.24
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           4.19e-18
Time:                           07:16:02   Log-Likelihood:                -888.95
No. Observations:                    408   AIC:                             1784.
Df Residuals:                        405   BIC:                             1796.
Df Model:                              2                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [165]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Income + Medical beds)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 1. 단어 순서대로 컬럼을 찾아서 리스트 생성
target_words = ['income', 'beds']

ordered_cols = [
    col for word in target_words
    for col in X_log_vars
    if word in col.lower()
]
# included_cols = [col for col in X_log_vars if any(w in col.lower() for w in target_words)]

# 2. 중복 제거 (한 컬럼에 여러 단어가 포함된 경우 대비)
seen = set()
included_cols = [c for c in ordered_cols if not (c in seen or seen.add(c))]

formula_X_log = ' + '.join(included_cols)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod2 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod2.summary())

# ------------------------------------------------------------------------------
# VIF 확인
# 1. 분석에 사용할 독립변수(X) 리스트 정의 (이전에 추출한 컬럼 리스트 활용)
# 예: filter_X_log_vars를 사용한다고 가정
X_data = df2_log[included_cols]

# 2. VIF 계산을 위해 상수항(intercept) 추가
# statsmodels의 VIF 공식은 상수항이 포함된 것을 전제로 합니다.
X_data_with_const = sm.add_constant(X_data)

# 3. VIF 계산
vif_Y1_mod2 = pd.DataFrame()
vif_Y1_mod2["feature"] = X_data_with_const.columns
vif_Y1_mod2["VIF"] = [variance_inflation_factor(X_data_with_const.values, i)
                   for i in range(len(X_data_with_const.columns))]

# 4. 결과 출력 (보통 상수항인 'const'는 제외하고 확인합니다)
print(vif_Y1_mod2[vif_Y1_mod2['feature'] != 'const'])

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.296
Model:                               OLS   Adj. R-squared:                  0.284
Method:                    Least Squares   F-statistic:                     24.06
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           2.58e-27
Time:                           07:16:11   Log-Likelihood:                -857.58
No. Observations:                    408   AIC:                             1731.
Df Residuals:                        400   BIC:                             1763.
Df Model:                              7                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [166]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Income + Medical beds + Medical staff)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 1. 단어 순서대로 컬럼을 찾아서 리스트 생성
target_words = ['income', 'beds', 'docs', 'nurses', 'staff']

ordered_cols = [
    col for word in target_words
    for col in X_log_vars
    if word in col.lower()
]
# included_cols = [col for col in X_log_vars if any(w in col.lower() for w in target_words)]

# 2. 중복 제거 (한 컬럼에 여러 단어가 포함된 경우 대비)
seen = set()
included_cols = [c for c in ordered_cols if not (c in seen or seen.add(c))]

formula_X_log = ' + '.join(included_cols)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod3 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod3.summary())

# ------------------------------------------------------------------------------
# VIF 확인
# 1. 분석에 사용할 독립변수(X) 리스트 정의 (이전에 추출한 컬럼 리스트 활용)
# 예: filter_X_log_vars를 사용한다고 가정
X_data = df2_log[included_cols]

# 2. VIF 계산을 위해 상수항(intercept) 추가
# statsmodels의 VIF 공식은 상수항이 포함된 것을 전제로 합니다.
X_data_with_const = sm.add_constant(X_data)

# 3. VIF 계산
vif_Y1_mod3 = pd.DataFrame()
vif_Y1_mod3["feature"] = X_data_with_const.columns
vif_Y1_mod3["VIF"] = [variance_inflation_factor(X_data_with_const.values, i)
                   for i in range(len(X_data_with_const.columns))]

# 4. 결과 출력 (보통 상수항인 'const'는 제외하고 확인합니다)
print(vif_Y1_mod3[vif_Y1_mod3['feature'] != 'const'])

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.408
Model:                               OLS   Adj. R-squared:                  0.394
Method:                    Least Squares   F-statistic:                     27.41
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           1.06e-39
Time:                           07:16:17   Log-Likelihood:                -822.15
No. Observations:                    408   AIC:                             1666.
Df Residuals:                        397   BIC:                             1710.
Df Model:                             10                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [167]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Income + Medical beds + Medical staff + Medical institutes)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 1. 단어 순서대로 컬럼을 찾아서 리스트 생성
target_words = ['income', 'beds', 'docs', 'nurses', 'staff',
                'hosp', 'hospital', 'clinic', 'health', 'infra']

ordered_cols = [
    col for word in target_words
    for col in X_log_vars
    if word in col.lower()
]
# included_cols = [col for col in X_log_vars if any(w in col.lower() for w in target_words)]

# 2. 중복 제거 (한 컬럼에 여러 단어가 포함된 경우 대비)
seen = set()
included_cols = [c for c in ordered_cols if not (c in seen or seen.add(c))]

formula_X_log = ' + '.join(included_cols)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod4 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod4.summary())

# ------------------------------------------------------------------------------
# VIF 확인
# 1. 분석에 사용할 독립변수(X) 리스트 정의 (이전에 추출한 컬럼 리스트 활용)
# 예: filter_X_log_vars를 사용한다고 가정
X_data = df2_log[included_cols]

# 2. VIF 계산을 위해 상수항(intercept) 추가
# statsmodels의 VIF 공식은 상수항이 포함된 것을 전제로 합니다.
X_data_with_const = sm.add_constant(X_data)

# 3. VIF 계산
vif_Y1_mod4 = pd.DataFrame()
vif_Y1_mod4["feature"] = X_data_with_const.columns
vif_Y1_mod4["VIF"] = [variance_inflation_factor(X_data_with_const.values, i)
                   for i in range(len(X_data_with_const.columns))]

# 4. 결과 출력 (보통 상수항인 'const'는 제외하고 확인합니다)
print(vif_Y1_mod4[vif_Y1_mod4['feature'] != 'const'])

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.718
Model:                               OLS   Adj. R-squared:                  0.706
Method:                    Least Squares   F-statistic:                     62.19
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           8.95e-97
Time:                           07:16:33   Log-Likelihood:                -671.11
No. Observations:                    408   AIC:                             1376.
Df Residuals:                        391   BIC:                             1444.
Df Model:                             16                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [168]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Income + Medical beds + Medical staff + Medical institutes + Older group)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 1. 단어 순서대로 컬럼을 찾아서 리스트 생성
target_words = ['income', 'beds', 'docs', 'nurses', 'staff',
                'hosp', 'hospital', 'clinic', 'health', 'infra',
                'elderly']

ordered_cols = [
    col for word in target_words
    for col in X_log_vars
    if word in col.lower()
]
# included_cols = [col for col in X_log_vars if any(w in col.lower() for w in target_words)]

# 2. 중복 제거 (한 컬럼에 여러 단어가 포함된 경우 대비)
seen = set()
included_cols = [c for c in ordered_cols if not (c in seen or seen.add(c))]

formula_X_log = ' + '.join(included_cols)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod5 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod5.summary())

# ------------------------------------------------------------------------------
# VIF 확인
# 1. 분석에 사용할 독립변수(X) 리스트 정의 (이전에 추출한 컬럼 리스트 활용)
# 예: filter_X_log_vars를 사용한다고 가정
X_data = df2_log[included_cols]

# 2. VIF 계산을 위해 상수항(intercept) 추가
# statsmodels의 VIF 공식은 상수항이 포함된 것을 전제로 합니다.
X_data_with_const = sm.add_constant(X_data)

# 3. VIF 계산
vif_Y1_mod5 = pd.DataFrame()
vif_Y1_mod5["feature"] = X_data_with_const.columns
vif_Y1_mod5["VIF"] = [variance_inflation_factor(X_data_with_const.values, i)
                   for i in range(len(X_data_with_const.columns))]

# 4. 결과 출력 (보통 상수항인 'const'는 제외하고 확인합니다)
print(vif_Y1_mod5[vif_Y1_mod5['feature'] != 'const'])

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.718
Model:                               OLS   Adj. R-squared:                  0.706
Method:                    Least Squares   F-statistic:                     58.53
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           4.98e-96
Time:                           07:16:49   Log-Likelihood:                -670.72
No. Observations:                    408   AIC:                             1377.
Df Residuals:                        390   BIC:                             1450.
Df Model:                             17                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

# 1-1차 Hierachical Model 결과 출력

In [169]:
file_path2 = '/content/drive/MyDrive/MedicalDA04/Semi Project#1/Analysis Dataset/3. 회귀 분석/'

# 0. OLS summary 확인
# Fetching SimpleTable objects
# Model 1 ~ 6 OLS summary
table_num0 = 0
tables_ori_log_mod1_0 = results_Y1_ori_log_mod1.summary().tables[table_num0]
tables_ori_log_mod2_0 = results_Y1_ori_log_mod2.summary().tables[table_num0]
tables_ori_log_mod3_0 = results_Y1_ori_log_mod3.summary().tables[table_num0]
tables_ori_log_mod4_0 = results_Y1_ori_log_mod4.summary().tables[table_num0]
tables_ori_log_mod5_0 = results_Y1_ori_log_mod5.summary().tables[table_num0]

table_num1 = 1
tables_ori_log_mod1_1 = results_Y1_ori_log_mod1.summary().tables[table_num1]
tables_ori_log_mod2_1 = results_Y1_ori_log_mod2.summary().tables[table_num1]
tables_ori_log_mod3_1 = results_Y1_ori_log_mod3.summary().tables[table_num1]
tables_ori_log_mod4_1 = results_Y1_ori_log_mod4.summary().tables[table_num1]
tables_ori_log_mod5_1 = results_Y1_ori_log_mod5.summary().tables[table_num1]

table_num2 = 2
tables_ori_log_mod1_2 = results_Y1_ori_log_mod1.summary().tables[table_num2]
tables_ori_log_mod2_2 = results_Y1_ori_log_mod2.summary().tables[table_num2]
tables_ori_log_mod3_2 = results_Y1_ori_log_mod3.summary().tables[table_num2]
tables_ori_log_mod4_2 = results_Y1_ori_log_mod4.summary().tables[table_num2]
tables_ori_log_mod5_2 = results_Y1_ori_log_mod5.summary().tables[table_num2]

# Model 1 ~ 6 IDV VIF summary
vif_log_mod1 = vif_Y1_mod1[vif_Y1_mod1['feature'] != 'const']
vif_log_mod2 = vif_Y1_mod2[vif_Y1_mod2['feature'] != 'const']
vif_log_mod3 = vif_Y1_mod3[vif_Y1_mod3['feature'] != 'const']
vif_log_mod4 = vif_Y1_mod4[vif_Y1_mod4['feature'] != 'const']
vif_log_mod5 = vif_Y1_mod5[vif_Y1_mod5['feature'] != 'const']


# 1. 저장할 파일 이름 정의
excel_file_name = f'{Y_var_name1}_Hierachical#1-1 OLS_VIF_Data_Sheets.xlsx'

# 2. 데이터프레임과 저장할 시트 이름을 딕셔너리로 매핑 (핵심)
# Convert SimpleTable objects to pandas DataFrames using as_html and read_html
data_to_save = {
    'OLS_log_mod1_Top': pd.read_html(tables_ori_log_mod1_0.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod2_Top': pd.read_html(tables_ori_log_mod2_0.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod3_Top': pd.read_html(tables_ori_log_mod3_0.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod4_Top': pd.read_html(tables_ori_log_mod4_0.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod5_Top': pd.read_html(tables_ori_log_mod5_0.as_html(), header=None, index_col=0)[0],

    'OLS_log_mod1_Tests': pd.read_html(tables_ori_log_mod1_2.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod2_Tests': pd.read_html(tables_ori_log_mod2_2.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod3_Tests': pd.read_html(tables_ori_log_mod3_2.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod4_Tests': pd.read_html(tables_ori_log_mod4_2.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod5_Tests': pd.read_html(tables_ori_log_mod5_2.as_html(), header=None, index_col=0)[0],

    'OLS_log_mod1_Coeffs': pd.read_html(tables_ori_log_mod1_1.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod2_Coeffs': pd.read_html(tables_ori_log_mod2_1.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod3_Coeffs': pd.read_html(tables_ori_log_mod3_1.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod4_Coeffs': pd.read_html(tables_ori_log_mod4_1.as_html(), header=None, index_col=0)[0],
    'OLS_log_mod5_Coeffs': pd.read_html(tables_ori_log_mod5_1.as_html(), header=None, index_col=0)[0],

    'OLS_log_mod1_VIF': vif_log_mod1,
    'OLS_log_mod2_VIF': vif_log_mod2,
    'OLS_log_mod3_VIF': vif_log_mod3,
    'OLS_log_mod4_VIF': vif_log_mod4,
    'OLS_log_mod5_VIF': vif_log_mod5
}

# 3. pandas.ExcelWriter 객체 생성 및 사용
# with 구문을 사용하면 파일 관리가 자동으로 처리되어 안전합니다.
with pd.ExcelWriter(file_path2 + excel_file_name, engine='xlsxwriter') as writer:
    # 딕셔너리를 순회하며 각 DataFrame을 시트별로 저장
    for sheet_name, dataframe in data_to_save.items():
        # index=True is used because we set index_col=0 when reading the HTML, making the first column the DataFrame index.
        dataframe.to_excel(writer, sheet_name=sheet_name, index=True)

/tmp/ipython-input-3240280788.py:41: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_log_mod1_Top': pd.read_html(tables_ori_log_mod1_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-3240280788.py:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_log_mod2_Top': pd.read_html(tables_ori_log_mod2_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-3240280788.py:43: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  'OLS_log_mod3_Top': pd.read_html(tables_ori_log_mod3_0.as_html(), header=None, index_col=0)[0],
/tmp/ipython-input-3240280788.py:44: FutureWarning: Passing literal html to 'read_html' 

# Backward Step Modeling 결과 (Backup)

In [ ]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Not included: log older pop)
# medical institute (Full) + medical staff (Full) + medical rooms (Full) + medical beds (Full) + income (Full)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 전체 컬럼에서 exclude_cols를 제외한 인덱스 반환
target_word = ['elderly']
exclude_cols = [col for col in X_log_vars if any(word in col.lower() for word in target_word)]
filter_X_log_vars = [col for col in X_log_vars if col not in exclude_cols]

formula_X_log = ' + '.join(filter_X_log_vars)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod1 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod1.summary())

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.801
Model:                               OLS   Adj. R-squared:                  0.790
Method:                    Least Squares   F-statistic:                     73.77
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):          5.34e-121
Time:                           03:09:26   Log-Likelihood:                -600.38
No. Observations:                    408   AIC:                             1245.
Df Residuals:                        386   BIC:                             1333.
Df Model:                             21                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [ ]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Not included: log all income_indiv)
# medical institute (Full) + medical staff (Full) + medical rooms (Full) + medical beds (Full) + income (Full)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 전체 컬럼에서 exclude_cols를 제외한 인덱스 반환
# 특정 단어가 포함된 컬럼 추출 (대소문자 구분 없이 처리하려면 .lower() 추가)
target_word = ['income']
exclude_cols = [col for col in X_log_vars if any(word in col.lower() for word in target_word)]
filter_X_log_vars = [col for col in X_log_vars if col not in exclude_cols]

formula_X_log = ' + '.join(filter_X_log_vars)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod2 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod2.summary())

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.562
Model:                               OLS   Adj. R-squared:                  0.539
Method:                    Least Squares   F-statistic:                     24.82
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           3.29e-57
Time:                           02:20:46   Log-Likelihood:                -760.88
No. Observations:                    408   AIC:                             1564.
Df Residuals:                        387   BIC:                             1648.
Df Model:                             20                                         
Covariance Type:               nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [ ]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Not included: log all medical beds)
# medical institute (Full) + medical staff (Full) + medical rooms (Full) + medical beds (Full) + income (Full)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 전체 컬럼에서 exclude_cols를 제외한 인덱스 반환
# 특정 단어가 포함된 컬럼 추출 (대소문자 구분 없이 처리하려면 .lower() 추가)
target_word = ['beds']
exclude_cols = [col for col in X_log_vars if any(word in col.lower() for word in target_word)]
filter_X_log_vars = [col for col in X_log_vars if col not in exclude_cols]

formula_X_log = ' + '.join(filter_X_log_vars)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod3 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod3.summary())

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.763
Model:                               OLS   Adj. R-squared:                  0.752
Method:                    Least Squares   F-statistic:                     73.71
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):          2.68e-110
Time:                           02:20:55   Log-Likelihood:                -635.88
No. Observations:                    408   AIC:                             1308.
Df Residuals:                        390   BIC:                             1380.
Df Model:                             17                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [ ]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Not included: log all medical rooms)
# medical institute (Full) + medical staff (Full) + medical rooms (Full) + medical beds (Full) + income (Full)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 전체 컬럼에서 exclude_cols를 제외한 인덱스 반환
# 특정 단어가 포함된 컬럼 추출 (대소문자 구분 없이 처리하려면 .lower() 추가)
target_word = ['rooms']
exclude_cols = [col for col in X_log_vars if any(word in col.lower() for word in target_word)]
filter_X_log_vars = [col for col in X_log_vars if col not in exclude_cols]

formula_X_log = ' + '.join(filter_X_log_vars)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod4 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod4.summary())

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.718
Model:                               OLS   Adj. R-squared:                  0.706
Method:                    Least Squares   F-statistic:                     58.53
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           4.98e-96
Time:                           02:21:00   Log-Likelihood:                -670.72
No. Observations:                    408   AIC:                             1377.
Df Residuals:                        390   BIC:                             1450.
Df Model:                             17                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [ ]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Not included: log all medical staff)
# medical institute (Full) + medical staff (Full) + medical rooms (Full) + medical beds (Full) + income (Full)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 전체 컬럼에서 exclude_cols를 제외한 인덱스 반환
# 특정 단어가 포함된 컬럼 추출 (대소문자 구분 없이 처리하려면 .lower() 추가)
target_word = ['docs', 'nurses', 'staff']
exclude_cols = [col for col in X_log_vars if any(word in col.lower() for word in target_word)]
filter_X_log_vars = [col for col in X_log_vars if col not in exclude_cols]

formula_X_log = ' + '.join(filter_X_log_vars)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod5 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod5.summary())

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.646
Model:                               OLS   Adj. R-squared:                  0.629
Method:                    Least Squares   F-statistic:                     37.31
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):           2.01e-75
Time:                           02:21:03   Log-Likelihood:                -717.27
No. Observations:                    408   AIC:                             1475.
Df Residuals:                        388   BIC:                             1555.
Df Model:                             19                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

In [ ]:
# Log Y1: 감염률 per100k (로그화) ~ 원시 독립변수(로그화)
# Model No.1 structure (Not included: log all medical institute)
# medical institute (Full) + medical staff (Full) + medical rooms (Full) + medical beds (Full) + income (Full)
X_log_vars = columns_log[2:] # log_tertiary_hosp ~

# 전체 컬럼에서 exclude_cols를 제외한 인덱스 반환
# 특정 단어가 포함된 컬럼 추출 (대소문자 구분 없이 처리하려면 .lower() 추가)
target_word = ['hosp', 'hospital', 'clinic', 'health', 'infra']
exclude_cols = [col for col in X_log_vars if any(word in col.lower() for word in target_word)]
filter_X_log_vars = [col for col in X_log_vars if col not in exclude_cols]

formula_X_log = ' + '.join(filter_X_log_vars)
formula_Y1_log = f"{Y_log_var_name1} ~ {formula_X_log}"
results_Y1_ori_log_mod6 = smf.ols(formula=formula_Y1_log, data=df2_log).fit()

print("### OLS 모델 결과 (Log Y1: 변화율 그대로) ###")
print(results_Y1_ori_log_mod6.summary())

### OLS 모델 결과 (Log Y1: 변화율 그대로) ###
                              OLS Regression Results                             
Dep. Variable:     log_confirmed_per100k   R-squared:                       0.728
Model:                               OLS   Adj. R-squared:                  0.717
Method:                    Least Squares   F-statistic:                     65.45
Date:                   Thu, 18 Dec 2025   Prob (F-statistic):          7.23e-100
Time:                           02:21:09   Log-Likelihood:                -663.57
No. Observations:                    408   AIC:                             1361.
Df Residuals:                        391   BIC:                             1429.
Df Model:                             16                                         
Covariance Type:               nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------